In [ ]:
import pandas as pd

In [ ]:
data1 = pd.read_csv("csc_threads.csv")[['id', 'author', 'selftext','created_utc', 'score']]

In [ ]:
data1['sub_id'] = data1['id']
data1['parent_id']='Null'

In [ ]:
data1.shape

In [ ]:
data1.head(5)

In [ ]:
data2 = pd.read_csv("csc_comm.csv")[['author', 'parent_id', 'sub_id','id', 'selftext','created_utc', 'score']]

In [ ]:
from datetime import datetime
data2['created_utc'] = data2['created_utc'].apply(lambda x: datetime.fromtimestamp(x).isoformat())

In [ ]:
data2['parent_id'] = data2['parent_id'].str.replace("t1_","")
data2['parent_id'] = data2['parent_id'].str.replace("t3_","")

In [ ]:
data2.shape

In [ ]:
data_combo = pd.concat([data1, data2])

In [ ]:
data_combo = data_combo.dropna(subset=['author'])

In [ ]:
data_combo.shape

In [ ]:
data_combo = data_combo.sort_values(by=['sub_id'])

In [ ]:
data_combo.head(5)

In [ ]:
id_author = dict(zip(data_combo['id'], data_combo['author']))
id_parent = dict(zip(data_combo['id'], data_combo['parent_id']))

In [ ]:
ids = data_combo['id'].to_list()

In [ ]:
# who replies to whom?
final_data = {'src':[], 'target':[]}
for i in ids:
    cur_parent = id_parent[i]
    cur_auth = id_author[i]
    if cur_parent != 'Null' and cur_parent in id_author.keys(): # if any rows are dropped their children are also dropped
        final_data['src'].append(cur_auth)
        parent_author = id_author[cur_parent]
        final_data['target'].append(parent_author)

In [ ]:
final_data = pd.DataFrame(final_data)

In [ ]:
final_data.head(20)

In [ ]:
final_data.shape

In [ ]:
#remove self loops
final_data = final_data[final_data['src']!=final_data['target']]

In [ ]:
final_data['weight'] = 1

In [ ]:
final_data.head(10)

In [ ]:
final_map = final_data.groupby(['src', 'target']).sum().reset_index()

In [ ]:
final_map.shape

In [ ]:
final_map = final_map.sort_values(by=['weight'])

In [ ]:
final_map[final_map['weight']>=2]

In [ ]:
final_map = final_map[final_map['src']!=final_map['target']]

In [ ]:
final_map['weight'] = final_map['weight'] /final_map['weight'].abs().max()

In [ ]:
final_map.shape

In [ ]:
final_map.head()

In [ ]:
final_map.to_csv("csc_2018_2021_halfsns.csv")

In [ ]:
import networkx as nx

In [ ]:
ggw_nx=nx.DiGraph()

In [ ]:
tupl = [tuple(r) for r in final_map.to_numpy().tolist()]

In [ ]:
ggw_nx.add_weighted_edges_from(tupl)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
nx.draw_spring(ggw_nx, with_labels = False)

In [ ]:
data_combo.shape

In [ ]:
data_combo.to_csv("csc_final.csv")